# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine

import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import numpy as np
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/mtx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mtx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mtx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
#df = pd.read_sql("SELECT * FROM InsertTableName", engine)
df = pd.read_sql_table('MessageAndLabel',engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.iloc[511].to_dict()

{'aid_centers': 0,
 'aid_related': 1,
 'buildings': 0,
 'child_alone': 0,
 'clothing': 0,
 'cold': 0,
 'death': 0,
 'direct_report': 1,
 'earthquake': 0,
 'electricity': 0,
 'fire': 0,
 'floods': 0,
 'food': 1,
 'genre': 'direct',
 'hospitals': 0,
 'id': 627,
 'infrastructure_related': 0,
 'medical_help': 0,
 'medical_products': 0,
 'message': 'We need Food, etc in Croix-des-Mission, Rail Street #17. Thank you for your understanding.',
 'military': 0,
 'missing_people': 0,
 'money': 0,
 'offer': 0,
 'original': 'Nou bezwen mange etc, se nan croix-des-missions rue rail impasse originale#17. MERCI pou bon konpreansyon nou..',
 'other_aid': 0,
 'other_infrastructure': 0,
 'other_weather': 0,
 'refugees': 0,
 'related': 1,
 'request': 1,
 'search_and_rescue': 0,
 'security': 0,
 'shelter': 0,
 'shops': 0,
 'storm': 0,
 'tools': 0,
 'transport': 0,
 'water': 0,
 'weather_related': 0}

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def pipeline_eval(Y_true,Y_pred):
    tmp_list = []
    for i,c in enumerate(Y_true.columns):
        accuracy = accuracy_score(Y_true[c], Y_pred[:,i])
        precision = precision_score(Y_true[c], Y_pred[:,i])
        recall = recall_score(Y_true[c], Y_pred[:,i])
        f1 = f1_score(Y_true[c], Y_pred[:,i])
        tmp_dict = {'feature':c,
                    'accuracy':accuracy,
                    'precision':precision,
                    'recall':recall,
                    'f1':f1
                   }
        tmp_list.append(tmp_dict)
        
    return pd.DataFrame(tmp_list).set_index('feature').sort_values('f1',ascending=False)

In [8]:
Y_train_pred = pipeline.predict(X_train)
eval_train = pipeline_eval(Y_train,Y_train_pred)
eval_train

,accuracy,f1,precision,recall
feature,,,,
related,0.991404,0.994418,0.991373,0.997482
aid_related,0.986064,0.982964,0.997602,0.968750
weather_related,0.987742,0.977558,0.998288,0.957672
earthquake,0.995066,0.972776,0.997123,0.949589
food,0.992828,0.966925,0.999031,0.936818
request,0.988454,0.965306,0.998735,0.934043
storm,0.993591,0.964785,0.997111,0.934488
shelter,0.993134,0.959251,1.000000,0.921694
direct_report,0.982503,0.952799,0.998275,0.911286


In [10]:
eval_train.describe().T[['mean','50%']]

,mean,50%
accuracy,0.993073,0.994405
f1,0.886775,0.913682
precision,0.970898,1.000000
recall,0.819684,0.841906


In [11]:
Y_test_pred = pipeline.predict(X_test)
eval0 = pipeline_eval(Y_test,Y_test_pred)
eval0#.describe().T[['mean','50%']]

/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
feature,,,,
related,0.788374,0.869434,0.821416,0.923415
earthquake,0.963839,0.789333,0.896970,0.704762
weather_related,0.835673,0.630785,0.835604,0.506608
aid_related,0.734056,0.618182,0.756568,0.522593
food,0.931797,0.602667,0.843284,0.468880
request,0.882514,0.516332,0.823647,0.376029
storm,0.927983,0.446009,0.742188,0.318792
direct_report,0.847574,0.433994,0.766000,0.302767
water,0.946445,0.304950,0.793814,0.188725


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_we

In [13]:
parameters = {'vect__min_df': [1, 5],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__max_depth':[3, 5],
              'clf__estimator__max_features': ['sqrt', 0.3]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [14]:
cv_model0 = cv.fit(X_train, Y_train)

In [15]:
cv_model0.cv_results_

{'mean_fit_time': array([ 12.36905201,  12.21069837,  17.0581336 ,  12.62557499,
         18.5510749 ,  16.40294727,  32.33455348,  27.24528964,
         10.66267761,  11.39202706,  12.09156632,  13.97115421,
         28.54757969,  23.36440905,  51.60794099,  40.25602706]),
 'mean_score_time': array([ 5.90295808,  4.75527859,  6.69831673,  5.07955829,  4.49791845,
         4.70792572,  5.2272532 ,  5.10411191,  4.72205035,  4.99968529,
         4.91412775,  5.01516151,  5.9167943 ,  5.24559037,  5.45099767,
         5.19548893]),
 'mean_test_score': array([ 0.20273638,  0.20110879,  0.20304155,  0.20268552,  0.20080362,
         0.20965363,  0.22058898,  0.21336656,  0.19993897,  0.19973552,
         0.20283811,  0.20131224,  0.22404761,  0.22358985,  0.23106658,
         0.2286252 ]),
 'mean_train_score': array([ 0.20309244,  0.20141398,  0.20314331,  0.20283814,  0.20591548,
         0.21860573,  0.2285235 ,  0.22066543,  0.20057476,  0.20110879,
         0.20304159,  0.20194805,  0.

In [16]:
cv_model0.best_params_

{'clf__estimator__max_depth': 5,
 'clf__estimator__max_features': 0.3,
 'clf__estimator__n_estimators': 25,
 'vect__min_df': 1}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
Y_test_pred = cv_model0.predict(X_test)
eval1 = pipeline_eval(Y_test,Y_test_pred)
eval1#.describe().T[['mean','50%']]

/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
feature,,,,
related,0.777540,0.871361,0.779725,0.987403
earthquake,0.969789,0.830479,0.901487,0.769841
food,0.954684,0.793607,0.797486,0.789765
water,0.965975,0.682788,0.813559,0.588235
floods,0.953616,0.637232,0.914384,0.489011
weather_related,0.841013,0.627591,0.894094,0.483480
shelter,0.943241,0.603412,0.813218,0.479661
storm,0.940494,0.599589,0.772487,0.489933
request,0.889075,0.565970,0.814433,0.433669


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
import pickle
pickle.dump(cv_model0, open('disaster_response_model.pkl', 'wb'))

In [22]:
from sklearn.externals import joblib
joblib.dump(cv_model0, 'disaster_response_model.m')

['disaster_response_model.m']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.